# Importar librerias y connección a BD

In [1]:
import os
import pymysql
import pandas as pd
import requests
import time
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from bs4 import BeautifulSoup
import csv
import time
import sys
import re

host = os.getenv('localhost')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')

conn = pymysql.connect(
    host = host,
    port = int(3306),
    user = "root",
    passwd ="",
    db = "recommender_system",
    charset = 'utf8mb4')



# [BD]Consulta en  si una lista esta registrada con metadata

In [2]:
def idImdbExists( idImdb ):    
    
    exist = True
    
    sql  = " SELECT * FROM lists WHERE idImdb = %s "
    
    with conn.cursor() as cursor:
    
        row_count = cursor.execute( sql, (idImdb) )    
        
        if row_count == 0:
        
            exist = False 
    
    return exist

# [BD] Comprueba si una lista está registrada en List_tittlelist

In [3]:
def idListExists( idImdb ):    
    
    exist = True
    
    sql  = " SELECT * FROM list_tittlelist WHERE id_list = %s "
    
    with conn.cursor() as cursor:
        
        row_count = cursor.execute( sql, ( idImdb ) )    
        
        if row_count == 0:
            
            exist = False 
    
    return exist

# [BD] Consulta si un tittle (pelicula) existe

In [4]:
def tittleExists( tconst ):
    
    exist = True
    
    sql  = " SELECT * FROM tittle_list WHERE tconst = %s "
    
    with conn.cursor() as cursor:
        
        row_count = cursor.execute( sql, ( tconst ) )    
        
        if row_count == 0:
            
            exist = False 
    
    return exist

# [BD] Insertar información descriptiva de una lista

In [5]:
def insertList( name, link, metadata, idImdb ):
    
    with conn.cursor() as cursor:
        
        # CREA UNA NUEVA FILA
        
        sql = "INSERT INTO lists ( name, link, metadata, idImdb ) VALUES ( %s, %s, %s,%s )"
        
        cursor.execute( sql,( name, link, metadata, idImdb ) )
        
        #GUARDAMOS LOS CAMBIOS
        
        conn.commit()
               
    

# [BD] Inserta asosiciaion pelicula-lista (tabla n:n)

In [6]:
def instertListTittleList( id_list, id_tittlelist):
    
    with conn.cursor() as cursor:
        
        # CREA UNA NUEVA FILA
        
        sql = "INSERT INTO list_tittlelist ( id_list, id_tittlelist ) VALUES ( %s, %s )"
        
        cursor.execute( sql,( id_list, id_tittlelist ) )
        
        #GUARDAMOS LOS CAMBIOS
        
        conn.commit()

# [BD] Inserta información de peliculas

In [7]:
def insertTittleList( tconst, f_created, f_modified, description, name_tittle, url, tittle_type, imdb_rating, runtime_mins, year, release_date):
    
    with conn.cursor() as cursor:
        
        # CREA UNA NUEVA FILA
        
        sql = "INSERT INTO tittle_list ( tconst, f_created, f_modified, description, name_tittle, url, tittle_type, imdb_rating, runtime_mins, year, release_date ) VALUES ( %s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s)"
        
        cursor.execute( sql,( tconst, f_created, f_modified, description, name_tittle, url, tittle_type, imdb_rating, runtime_mins, year, release_date ) )
        
        
        #GUARDAMOS LOS CAMBIOS
        
        conn.commit()

# [BD] OBTENGO TODAS LAS PELICULAS DE UNA LISTA

In [8]:
def getTitlesList(idList):
    
    sql  = " SELECT * FROM list_tittlelist WHERE id_list = %s "
    
    with conn.cursor() as cursor:
        
        result = cursor.execute( sql, ( idList ) )    
        
        if result == 0:
            
            return -1
        
        result= cursor.fetchall()
    
    return result

# [BD] Buscar si una pelicula dentro de una lista existe

In [9]:
def titleExistInList( idList, idTitleList ):
    
    exist = True
    
    sql  = "SELECT * FROM list_tittlelist where id_list=%s AND id_tittlelist=%s"
    
    with conn.cursor() as cursor:
        
        row_count = cursor.execute( sql, ( idList, idTitleList ) )    
        
        if row_count == 0:
            
            exist = False 
    
    return exist


# [IMDB] Numero de consultas(peticiones) que se ha hecho

In [10]:
def getNIterations( number ):
    
    nIteration = 1
    div = number%100
   
    if number >= 100:
        
        nIteration = int( number/100 )+1
      
        
    return nIteration

# [IMDB] Se obtiene informacion referente a una lista

In [11]:
def getInfoLists( lists ):
    
    resultList = []
    
    #RECORRO LAS LISTAS Y SACO LA INFORMACIÓN
    
    for list in lists:
        
        collectorList = []
        
        #OBTENGO EL NOMBRE DE LA LISTA
        
        infoList = list.find( "div" , {"class": "list_name"} )
        
        nameList = infoList.find( "a" )    
        
        collectorList.append( nameList.text )


        #OBTENGO EL LINK DE LA LISTA

        linkList = infoList.find( 'a', href = True) 
        
        collectorList.append( linkList[ 'href' ] )
        


        #OBTENGO LA METADATA DE LA LISTA
        
        metadataList = list.find( "div" , {"class": "list_meta"} )
        
        collectorList.append( metadataList.text )

        resultList.append( collectorList )
        
    return resultList

# [IMDB] Se modifica la URL para modificar a traves de las paginas

In [12]:
def formatUrlPage( url, number ):
    
    url += "?page=%d"%number #url formatada para rec
    
    return  url
    

# [IMDB] Se descarga excel que contiene el listado de peliculas de una lista

In [13]:
def downloadExcel( urlIdList, urlDomain = "https://www.imdb.com/list/" ):
    
    # input urlIdList: string con (1)url de una lista completa o (2)solo el ID de una lista || 
    
    #           ej(1): "https://www.imdb.com/list/ls092155333/"
    
    #           ej(2): "list/ls092083481"   
    
    #       urlDomain(opcional): string con el nombre del dominio de la pagina || ej: "https://www.imdb.com/"
    
    # return fileName: string referencia al nombre del excel que tiene el listado de peliculas || ej: "ls092083481.csv"
    
    
    match = re.search( "^https.", urlIdList ) #busca expresión regular "https" para identificar si es un url completa o solo el id
    
    if match: 
        
        #contiene http, por lo tanto es url completa
        
        stringExport = "export?ref_=ttls_otexp" #string con parametro que le indica al sitio descargar excel
        
        url = urlIdList + stringExport
        
        print("\n Se descargará url con http: ", url)
        
        idList = urlIdList.split( "/", 5 )[-2]
        
        
    else:
        
        #sin http, es decir url relativa con referencia al ID de la lista
        
        #TODO: OPTIMIZAR Y DEJAR OPCION PARA QUE INGRESEN SOLO EL ID DE LA LISTA "ls092155333"
        
        stringExport = "/export?ref_=ttls_otexp"
        
        url = urlDomain + urlIdList + stringExport
        
        print("\n Se descargará url sin http: ", url )
        
        idList = urlIdList.split( "/", 2 )[-1] 
    
    time.sleep( 1 )
    
    http = urllib3.PoolManager()
    
    r = http.request('GET', url, preload_content=False)
    
    if r.status == 200:
        
        print("\n Descargando excel de lista [%s]..." %( idList ) )
        
        fileName= idList+".csv"
        
        with open( fileName, 'wb' ) as out:
            
            while True:
                
                data = r.read()
                
                if not data:
                    
                    break
                
                out.write( data )
    else:
        
        print("\n NO SE PUDO DESCARGAR LISTA [%s] STATUS CODE [%s]"%( idList, r.status ) )
        
        return False
    
    r.release_conn()
    
    print("\n Lista [%s] se guardó su excel con nombre [%s]"%( idList, fileName ) )
    
    return fileName

# [IMDB] Función que comienza el proceso de descargar listados de peliculas relacionadas al ID de una pelicula

In [14]:


def getListImdb( url ):
    
    #Input url = https://www.imdb.com/lists/tt0074486?ref_=tt_rls_sm
    
    #return lista= con metadatos [0]-> [Nombre, idList, metadatos]
    
    #Esta Función descarga todas las listas asociadas al ID de una película. Navega por el dominio de IMDB
    
    countPage = 1 #contador que irá aumentando para buscar más páginas con listas asociadas a la pelicula buscada. Minimo debe ser 1
    
    url = formatUrlPage( url, countPage ) #string de la URL que se le concatena el número de páginas
    
    r = requests.get( url ) #obtiene la petición a la URL
    
    html_content = r.text #se "parsea" la petición a string (texto)
    
    soup = BeautifulSoup( html_content, 'lxml' ) #se crea una instancia de BeatifulSoup, librería usada para navegar por el DOM HTML
    
    nIterationsText =  soup.find( "span", {"class" : "list-pagination-total-elements" } ) #se busca en el DOM la cantidad de páginas totales
    
    nIterations = getNIterations( int( nIterationsText.text ) )   #se "parsea" el número obtenido anteriormente a un valor tipo Entero 
    
    countIterations = 0 #contador que irá indicando las iteraciones (páginas) que se encuentra posicionado en la función
    
    resultList = None # colección que guardará la información extraidas de las páginas
    
    countNot200 = 0 #contador que indicará el límite de peticiones rechazadas por IMDB
    
    #while countIterations < nIterations:
    
    while countIterations < 1:
        
        time.sleep(2) # se da una pausa de 2 segundo, para no saturar de peticiones a IMDB
         
            
        if r.status_code == 200:
            
            countNot200 = 0
            
            html_content = r.text
            
            soup = BeautifulSoup( html_content, 'lxml' )
            
            #OBTENGO TODAS LAS LISTAS
            
            lists = soup.findAll( "div", { "class": "list-preview" } ) #se obtiene del DOM metadata asociada a la listas 
            
            resultList = getInfoLists( lists )
            
            countPage += 1
            
            url = url.split( '?',1 )[0]
            
            url = formatUrlPage( url, countPage )
            
            r = requests.get( url )
            
            countIterations += 1

        elif countNot200 >= 2:
            
            print ( "ERROR AL DESCARGAR LISTAS: Se guardaron solo %d" , countIterations )
            
            return resultList
        
        else:
            
            countNot200 += 1
    
    print ( "Se guardaron bien todas las  listas" )    
    
    return resultList


# [IMDB] Proceso de rescatar los datos del excel descargado

In [15]:
def readCsvAndSaved( fileName ):

    # input fileName: referencia al nombre de la lista que se leerá en excel || ej: ls080428597
    
    print("\n Comienza proceso de leer excel[%s] y guardar su metadata..."%( fileName ) )
    
    if fileName == False:
        
        print("\n No se puede leer archivo[%s]"%( fileName ) )
        
        return fileName
    
    if (fileName.find('.csv') == -1):
        
        idList = fileName
        
        fileName += ".csv"
    
    else:
        
        idList = fileName.split( '.', 1 )[0]
    
    with open( fileName, 'r' ) as file:
        
        reader = csv.reader( file, delimiter = ',' )
        
        countRow = 0
        
        print("\n Leyendo archivo [%s] y guarando su información..."%( fileName ) )
        
        for row in reader:
            
            if countRow != 1 :
                
                if not ( tittleExists( row[1] ) ):
                    
                    instertListTittleList( idList, row[1] )
                    
                    insertTittleList( row[1], row[2], row[3], row[4] ,row[5], row[6], row[7], row[8], row[9], row[10], row[13] )                  
                
                
            countRow += 1
            
        print( "\n [%s] Registros guardados"%( countRow ) )
        
    return True

# [IMDB] Compara peliculas de una lista en una lista dada

In [16]:
def listCompare( idOriginList , idEndList ):
    
    print("\n **********************************************************")
    
    print("\n Comenzando proceso de comparación")
    
    print("\n idOriginList[%s] idEndList[%s]" % ( idOriginList, idEndList ) )
    
    originTitles = getTitlesList( idOriginList )
    
    result = []
    
    numIterator = 1
    
    numFind = 0
    
    if isinstance( idEndList , list ):
        
                
        print( "\n Es LISTA" )
        
        numIterator = len( idEndList )        
        
        
        
        print( "\n NUM ITERATOR: ", numIterator)
        
        for i in range( numIterator ):
            
            auxMatrix = []
            
            for oTitle in originTitles:        
                
                print( "\n idEndList[%s] oTitle[%s] " % ( idEndList[ i ], oTitle[ 2 ] ) )
                
                if titleExistInList( idEndList[ i ] , oTitle[ 2 ] ):
                    
                    print( "\n ENCONTRADO" )
                    
                    auxMatrix.append( 1 )
                    
                    numFind += 1
                    
                else:           
                    
                    #print( "\n NO ENCONTRADO" )
                    
                    auxMatrix.append( 0 )
                    
            result.append( auxMatrix )
            
            auxMatrix.clear()
            
            auxMatrix = None
    else:
         
            for oTitle in originTitles:        
                
                if titleExistInList( idEndList, oTitle[ 2 ] ):
                    
                    result.append( 1 )
                    
                    numFind += 1
                    
                else:           
                    
                    result.append( 0 )
    
    print("\n Resultado de la comparación: \n Numero de tittles coincidentes[%s] \n Matriz: %s" % ( numFind, result ) )
    
    print("\n **********************************************************")
    
    return result

# [IMDB] Leer excel y guardar su contenido en BD

In [17]:
def saveDataList( idList ):
    
    flag = False
    
    fileNameList = downloadExcel( idList )
    
    if fileNameList != False:
        
        dataCsv = readCsvAndSaved( fileNameList )
        
        if dataCsv != False:
            
            flag = True
            
            print( "\n Lista[%] se guardó con éxtio" % ( idList ) )
    
    return flag

# [IMDB] Proceso de selección del tipo de descarga

In [18]:
def startMetadataListProcess( originList, tipoDescarga, endList = None ):
    
    print("\n Comenzando comprobación y descarga de información OriginList[%s] tipoDescarga[%s]" % ( originList, tipoDescarga ) )
    
    if  tipoDescarga == 1:
        
        print("\n Procesando descarga de una lista directamente...")
        
        #idOriginList=originList.split('/',1)[1]
        
        idOriginList=originList
        
        #TODO: OPTIMIZAR ESTE METODO
        
        if not idListExists( idOriginList ):
            
            print("\n Descargando lista de origen[%s] " % ( idOriginList) )
            
            saveDataList( originList )
        
        for lista in endList:  
           
            #idEndList=lista.split('/',1)[1]
            
            idEndList=lista
            
            print("\nComenzando descarga de endList[%s] " % ( idEndList ) )
            
            if not idListExists(idEndList): 
                
                saveDataList(idEndList)
            
            else: print( "Estaba registrada en BD" )
    else:
        
        listas = getListImdb( originList )
        
        toolbar_width = 40
        
        totalIteration = len( listas )
        
        countAdvance = 1
        
        # setup toolbar
        
        sys.stdout.write( "[%s]" % (" " * toolbar_width ) )
        
        #sys.stdout.flush()
        
        sys.stdout.write( "\b" * ( toolbar_width +1 ) ) # return to start of line, after '['
        
        
        #TODO: DEJAR ESTE PRCESO EN UN METODO APARTE
        
        for lista in listas:
            
            # update the bar
            
            sys.stdout.write( "\r %d de %d" % ( countAdvance, totalIteration ) )

            countAdvance += 1
            
            sys.stdout.flush()
            
            time.sleep( 1 )
            
            idImdb = lista[ 1 ]
            
            idImdb = idImdb.split( '/', 2 )[-1]        

            if not ( idImdbExists( lista[1] ) ):    

                fileNameList = downloadExcel( lista[1] )            

                dataCsv= readCsvAndSaved( idImdb )


                insertList( lista[0], lista[1], lista[2], idImdb )
        
        sys.stdout.write( "]\n" ) # this ends the progress bar
    
       
    print( "\n ¡Proceso finalizado!" )

# ================ Lógica de Bayes ========================

# Función principal que inicia todo los procesos ( no terminada)

In [19]:


    listas = "https://www.imdb.com/list/ls096276650/?sort=alpha,asc&st_dt=&mode=detail&page=1"
    endList = []
    endList.append("ls092155333")#L1
    endList.append("ls094566741")#L2
    endList.append("ls096276650") #L3
    endList.append("ls021428038")
    originList="ls096276650"
  
    #listCompare("ls063431068","ls063363649")
    startMetadataListProcess(originList,1, endList)
    listCompare(originList,endList)



 Comenzando comprobación y descarga de información OriginList[ls096276650] tipoDescarga[1]

 Procesando descarga de una lista directamente...

Comenzando descarga de endList[ls092155333] 
Estaba registrada en BD

Comenzando descarga de endList[ls094566741] 
Estaba registrada en BD

Comenzando descarga de endList[ls096276650] 
Estaba registrada en BD

Comenzando descarga de endList[ls021428038] 
Estaba registrada en BD

 ¡Proceso finalizado!

 **********************************************************

 Comenzando proceso de comparación

 idOriginList[ls096276650] idEndList[['ls092155333', 'ls094566741', 'ls096276650', 'ls021428038']]

 Es LISTA

 NUM ITERATOR:  4

 idEndList[ls092155333] oTitle[tt5503686] 

 idEndList[ls092155333] oTitle[tt8722346] 

 idEndList[ls092155333] oTitle[tt0104057] 

 idEndList[ls092155333] oTitle[tt0086619] 

 idEndList[ls092155333] oTitle[tt0084269] 

 idEndList[ls092155333] oTitle[tt0364930] 

 idEndList[ls092155333] oTitle[tt1239285] 

 idEndList[ls09215

[[], [], [], []]